In [ ]:
import pandas as pd
import numpy as np
import cudf
import cupy as cp
import optuna
import lightgbm as lgb
from cuml.svm import SVR as cumlSVR
from cuml.linear_model import ElasticNet as cuElasticNet
from cuml.ensemble import RandomForestRegressor as cuRF
import torch
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

# ------------------------- Data load and preprocessing -------------------------
df = pd.read_excel("/home/pumc/tangrui_zy/1F咽拭子-去重-计算A+C.xlsx")  
X = df[["Month", "Day", "Hour", "the day of week", "the number of queuing patient", "Arrival rate"]].values.astype('float32')
y = df["Queuing time"].values.astype('float32')

X_cudf = cudf.DataFrame.from_records(X)
y_cudf = cudf.Series(y)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# ----------------------------- LightGBM-GPU --------------------------------
def objective_lgb(trial):
    params = {
        'objective': 'regression',
        'metric': 'root_mean_squared_error',
        'device': 'gpu',
        'verbosity': -1,
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 15, 100),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
    }
    rmse_list = []
    for train_idx, val_idx in kf.split(X):
        train_data = lgb.Dataset(X[train_idx], label=y[train_idx])
        val_data = lgb.Dataset(X[val_idx], label=y[val_idx])
        model = lgb.train(params, train_data, valid_sets=[val_data])
        y_pred = model.predict(X[val_idx])
        rmse_list.append(root_mean_squared_error(y[val_idx], y_pred))
    return np.mean(rmse_list)

study_lgb = optuna.create_study(direction='minimize')
study_lgb.optimize(objective_lgb, n_trials=50)
best_params_lgb = study_lgb.best_params

model_lgb = lgb.LGBMRegressor(**best_params_lgb, device='gpu')
model_lgb.fit(X, y)
preds_lgb = model_lgb.predict(X)

In [ ]:
# ----------------------------- cuML SVR --------------------------------
def objective_svr(trial):
    C = trial.suggest_float('C', 0.01, 100, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-3, 1.0, log=True)
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    rmse_list = []
    for train_idx, val_idx in kf.split(X):
        svr = cumlSVR(C=C, epsilon=epsilon, gamma=gamma)
        svr.fit(X_cudf.iloc[train_idx], y_cudf.iloc[train_idx])
        preds = svr.predict(X_cudf.iloc[val_idx])
        rmse_list.append(root_mean_squared_error(y[val_idx], cp.asnumpy(preds)))
    return np.mean(rmse_list)

study_svr = optuna.create_study(direction='minimize')
study_svr.optimize(objective_svr, n_trials=50)
best_params_svr = study_svr.best_params

model_svr = cumlSVR(**best_params_svr)
model_svr.fit(X_cudf, y_cudf)
preds_svr = cp.asnumpy(model_svr.predict(X_cudf))

In [ ]:
# ----------------------------- cuML ElasticNet ------------------------------
def objective_elastic(trial):
    alpha = trial.suggest_float("alpha", 1e-4, 10.0, log=True)
    l1_ratio = trial.suggest_float("l1_ratio", 0.0, 1.0)
    rmse_list = []
    for train_idx, val_idx in kf.split(X):
        X_train = X_cudf.iloc[train_idx].reset_index(drop=True)
        X_val = X_cudf.iloc[val_idx].reset_index(drop=True)
        y_train = y_cudf.iloc[train_idx].reset_index(drop=True)
        y_val = y_cudf.iloc[val_idx].reset_index(drop=True)
        model = cuElasticNet(alpha=alpha, l1_ratio=l1_ratio)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = root_mean_squared_error(cp.asnumpy(y_val.values), cp.asnumpy(preds))
        rmse_list.append(rmse)
    return np.mean(rmse_list)

study_elastic = optuna.create_study(direction="minimize")
study_elastic.optimize(objective_elastic, n_trials=50)
best_params_elastic = study_elastic.best_params

model_elastic = cuElasticNet(alpha=best_params_elastic["alpha"], l1_ratio=best_params_elastic["l1_ratio"])
model_elastic.fit(X_cudf, y_cudf)
preds_elastic = cp.asnumpy(model_elastic.predict(X_cudf))

In [ ]:
# ----------------------------- XGBoost --------------------------------
def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 0.5),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0),
        "tree_method": "hist",  
        "random_state": 0
    }

    rmse_list = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = XGBRegressor(**params)
        model.fit(X_train, y_train, verbose=False)
        preds = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, preds)
        rmse_list.append(rmse)
    return np.mean(rmse_list)

study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=50, timeout=600)
best_params_xgb = study_xgb.best_params

model_xgb = XGBRegressor(**best_params_xgb)
model_xgb.fit(X, y)
preds_xgb = model_xgb.predict(X)

In [ ]:
# ----------------------------- cuML Random Forest ------------------------------
def objective_rf(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 10, 40),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
        "max_features": trial.suggest_float("max_features", 0.3, 1.0),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False])
    }

    val_rmse_scores = []
    for train_idx, val_idx in kf.split(X):
        X_train = X_cudf.iloc[train_idx].reset_index(drop=True)
        X_val = X_cudf.iloc[val_idx].reset_index(drop=True)
        y_train = y_cudf.iloc[train_idx].reset_index(drop=True)
        y_val = y_cudf.iloc[val_idx].reset_index(drop=True)
        model = cuRF(**params)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = root_mean_squared_error(cp.asnumpy(y_val.values), cp.asnumpy(preds))
        val_rmse_scores.append(rmse)
    return np.mean(val_rmse_scores)

study_rf = optuna.create_study(direction="minimize")
study_rf.optimize(objective_rf, n_trials=50)
best_params_rf = study_rf.best_params

model_rf = cuRF(**best_params_rf)
model_rf.fit(X_cudf, y_cudf)
preds_rf = cp.asnumpy(model_rf.predict(X_cudf))

In [ ]:
# ----------------------------- Decision Tree --------------------------------
def objective_dt(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
        "max_features": trial.suggest_float("max_features", 0.3, 1.0)
    }

    val_rmse_scores = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = DecisionTreeRegressor(**params)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, preds)
        val_rmse_scores.append(rmse)
    return np.mean(val_rmse_scores)

study_dt = optuna.create_study(direction="minimize")
study_dt.optimize(objective_dt, n_trials=50)
best_params_dt = study_dt.best_params

model_dt = DecisionTreeRegressor(**best_params_dt)
model_dt.fit(X, y)
preds_dt = model_dt.predict(X)

In [ ]:
# ----------------------------- KNeighborsRegressor --------------------------------
def objective_knn(trial):
    params = {
        "n_neighbors": trial.suggest_int("n_neighbors", 3, 20),
        "weights": trial.suggest_categorical("weights", ["uniform", "distance"]),
        "p": trial.suggest_int("p", 1, 2)  # 1: Manhattan distance, 2: Euclidean distance
    }

    val_rmse_scores = []
    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = KNeighborsRegressor(**params)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, preds)
        val_rmse_scores.append(rmse)
    return np.mean(val_rmse_scores)

study_knn = optuna.create_study(direction="minimize")
study_knn.optimize(objective_knn, n_trials=50)
best_params_knn = study_knn.best_params

model_knn = KNeighborsRegressor(**best_params_knn)
model_knn.fit(X, y)
preds_knn = model_knn.predict(X)

In [ ]:
# ----------------------------- Bootstrap evaluation --------------------------------
def bootstrap_evaluate(y_true, y_pred, n_bootstrap=1000):
    mse_list, rmse_list, mae_list, r2_list = [], [], [], []
    for _ in range(n_bootstrap):
        indices = np.random.choice(len(y_true), len(y_true), replace=True)
        y_true_boot = y_true[indices]
        y_pred_boot = y_pred[indices]
        mse_list.append(mean_squared_error(y_true_boot, y_pred_boot))
        rmse_list.append(root_mean_squared_error(y_true_boot, y_pred_boot))
        mae_list.append(mean_absolute_error(y_true_boot, y_pred_boot))
        r2_list.append(r2_score(y_true_boot, y_pred_boot))
    return {
        "MSE": (np.mean(mse_list), np.std(mse_list)),
        "RMSE": (np.mean(rmse_list), np.std(rmse_list)),
        "MAE": (np.mean(mae_list), np.std(mae_list)),
        "R2": (np.mean(r2_list), np.std(r2_list))
    }

# Evalution for all models
models = {
    "LightGBM": preds_lgb,
    "SVR": preds_svr,
    "ElasticNet": preds_elastic,
    "XGBoost": preds_xgb,
    "RandomForest": preds_rf,
    "DecisionTree": preds_dt,
    "KNeighborsRegressor": preds_knn
}

results = {}
for name, preds in models.items():
    results[name] = bootstrap_evaluate(y, preds)

# Print result
for name, metrics in results.items():
    print(f"Model: {name}")
    for metric, (mean, std) in metrics.items():
        print(f"{metric}: {mean:.4f} ± {std:.4f}")
    print("-" * 30)

# ----------------------------- Output of the best hyperparameter summary --------------------------------
print("LightGBM 最优超参数:", best_params_lgb)
print("SVR 最优超参数:", best_params_svr)
print("ElasticNet 最优超参数:", best_params_elastic)
print("XGBoost 最优超参数:", best_params_xgb)
print("RandomForest 最优超参数:", best_params_rf)
print("DecisionTree 最优超参数:", best_params_dt)
print("KNeighborsRegressor 最优超参数:", best_params_knn)